# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [186]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [188]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,mili,6.0815,171.7350,29.12,76,54,6.51,MH,1724701081
1,1,adamstown,-25.0660,-130.1015,18.65,73,100,7.81,PN,1724701082
2,2,ugoofaaru,5.6667,73.0000,27.85,73,39,3.51,MV,1724701083
3,3,stanley,54.8680,-1.6985,15.01,79,75,3.09,GB,1724701084
4,4,whitehorse,60.7161,-135.0538,11.56,61,75,2.06,CA,1724701085


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [191]:
# Configure the map plot
cities_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 2,
    color = "City",
    line_color='black'
)

# Display the map
cities_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [194]:
# Create a copy of the above dataframe to avoid dropna error later
city_copy = city_data_df.copy()

# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_copy[(city_copy["Max Temp"] > 17) &\
    (city_copy["Max Temp"] < 23) &\
    (city_copy["Humidity"] < 70) &\
    (city_copy["Wind Speed"] < 4.5) &\
    (city_copy["Cloudiness"] < 50)\
    ]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
101,101,scottburgh,-30.2867,30.7532,20.19,51,49,2.23,ZA,1724701200
105,105,jinchang,38.4953,102.1739,19.09,54,9,1.78,CN,1724701206
122,122,ambovombe,-25.1667,46.0833,20.49,66,0,1.99,MG,1724701224
186,186,chudniv,50.0520,28.1175,22.58,33,6,3.63,UA,1724701295
188,188,kolpny,52.2282,37.0368,20.85,49,0,3.21,RU,1724701297
226,226,yefremov,53.1481,38.0992,20.70,56,0,2.85,RU,1724701339
227,227,la ronge,55.1001,-105.2842,18.12,42,0,2.06,CA,1724701340
248,248,south hedland,-20.4089,118.5983,17.96,45,14,4.12,AU,1724701367
263,263,shubarshi,48.5872,57.1869,22.66,50,49,3.97,KZ,1724701385
342,342,port saint john's,-31.6229,29.5448,20.91,45,3,2.89,ZA,1724701476


### Step 3: Create a new DataFrame called `hotel_df`.

In [197]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df.copy()
hotel_df = hotel_df[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ["" for _ in range(len(hotel_df))]

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
101,scottburgh,ZA,-30.2867,30.7532,51,
105,jinchang,CN,38.4953,102.1739,54,
122,ambovombe,MG,-25.1667,46.0833,66,
186,chudniv,UA,50.0520,28.1175,33,
188,kolpny,RU,52.2282,37.0368,49,
226,yefremov,RU,53.1481,38.0992,56,
227,la ronge,CA,55.1001,-105.2842,42,
248,south hedland,AU,-20.4089,118.5983,45,
263,shubarshi,KZ,48.5872,57.1869,50,
342,port saint john's,ZA,-31.6229,29.5448,45,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [230]:
# Set the parameters for the type of place
categories = "accommodation.hotel"
radius = 10000

# Set the parameters for the type of search
filters = f"circle:{longitude},{latitude},{radius}"
bias = f"proximity:{longitude},{latitude}"
limit = 20

# set up a parameters dictionary
params = {
    "categories":categories,
    "limit":limit,
    "filter":filters,
    "bias":bias,
    "apiKey":geoapify_key    
}

# Set base URL
base_url = "https://api.geoapify.com/v2/places"

# Set the geographical coordinates for Sydney, Australia
latitude = 42.728042349999996
longitude = -88.9929183092243

# run a request using our params dictionary
response = requests.get(base_url, params=params)

# convert response to json
hotels_data = response.json()


print(f"{hotels_data["features"][0]["properties"]["address_line1"]};  {hotels_data["features"][0]["properties"]["address_line2"]}")
# Print the json (pretty printed)
#print(json.dumps(places_data, indent=4, sort_keys=True))


Hampton Inn & Suites;  2400 Fulton Street, Janesville, WI 53546, United States of America


In [234]:
# Set base URL 
base_url = "https://api.geoapify.com/v2/places"

# Set the initially known parameter variables for the search
categories = "accommodation.hotel"
radius = 10000
limit = 20

# Set parameters to search for a hotel
params = {
    "categories":categories,
    "limit":limit,
    "filter":filters,
    "bias":bias,
    "apiKey":geoapify_key    
    }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():

    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index,"Lat"]
    longitude = hotel_df.loc[index,"Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
scottburgh - nearest hotel: Blue Marlin Hotel
jinchang - nearest hotel: No hotel found
ambovombe - nearest hotel: No hotel found
chudniv - nearest hotel: No hotel found
kolpny - nearest hotel: No hotel found
yefremov - nearest hotel: Счастье
la ronge - nearest hotel: La Ronge Hotel and Suites
south hedland - nearest hotel: Port Haven
shubarshi - nearest hotel: No hotel found
port saint john's - nearest hotel: Outback Inn
sivas - nearest hotel: Sultan Otel
illapel - nearest hotel: Domingo Ortiz de Rosas
la punta - nearest hotel: La Posada de Salim
colonia - nearest hotel: Wasserturm Hotel Cologne
tsiombe - nearest hotel: No hotel found
kalemie - nearest hotel: Hotel du Lac
saurimo - nearest hotel: Hotel Princesinha
mil'kovo - nearest hotel: Дрозды
caconda - nearest hotel: No hotel found
mount isa - nearest hotel: Ibis Styles
rakops - nearest hotel: Rakops River Lodge


,City,Country,Lat,Lng,Humidity,Hotel Name
101,scottburgh,ZA,-30.2867,30.7532,51,Blue Marlin Hotel
105,jinchang,CN,38.4953,102.1739,54,No hotel found
122,ambovombe,MG,-25.1667,46.0833,66,No hotel found
186,chudniv,UA,50.0520,28.1175,33,No hotel found
188,kolpny,RU,52.2282,37.0368,49,No hotel found
226,yefremov,RU,53.1481,38.0992,56,Счастье
227,la ronge,CA,55.1001,-105.2842,42,La Ronge Hotel and Suites
248,south hedland,AU,-20.4089,118.5983,45,Port Haven
263,shubarshi,KZ,48.5872,57.1869,50,No hotel found
342,port saint john's,ZA,-31.6229,29.5448,45,Outback Inn


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [242]:
# Configure the map plot
hotels_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 2,
    color = "City",
    line_color='black',
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
hotels_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)